In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! pip install git+https://github.com/LIAAD/yake
import yake

  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-cbpn_xlw
  Running command git clone -q https://github.com/LIAAD/yake /tmp/pip-req-build-cbpn_xlw


In [3]:
from collections import defaultdict
from math import log, sqrt
import re
import numpy as np
import sys
from copy import deepcopy
import pandas as pd
import numpy as np
import csv
import json
from itertools import islice
from collections import OrderedDict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import nltk
from glob import glob
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from tqdm import tqdm
import pickle
import math
import re
from sklearn.model_selection import train_test_split
import operator
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import datetime
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('corpus')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Error loading corpus: Package 'corpus' not found in index


False

In [4]:
file1 = open("/content/drive/MyDrive/Tweelink_Dataset/twitter_base_preprocessed.pkl", "rb")
df = pickle.load(file1)
file1.close()

In [5]:
def get_relevant_docs_list_for_base_hashtag(base_hashtag, base_date, docs_preprocessed):
  relevant_docs_list = []
  for doc in docs_preprocessed:
    if doc['Data']['Base Hashtag']==base_hashtag:
      current_date = datetime.datetime.strptime(base_date, format)
      prev_date = current_date - datetime.timedelta(days=1)
      next_date = current_date + datetime.timedelta(days=1)
      if(doc['Data']['Date'] in [str(prev_date.date()), str(current_date.date()), str(next_date.date())]):
        relevant_docs_list.append(doc['Name'])
  return relevant_docs_list

In [6]:
def precision_at_k(k, base_hashtag, base_date, prediction_list, docs_preprocessed):
  relevant_docs_list = get_relevant_docs_list_for_base_hashtag(base_hashtag, base_date, docs_preprocessed)
  num_of_relevant_results=0
  for itr in range(k):
    if (prediction_list[itr] in relevant_docs_list):
      num_of_relevant_results+=1
  return num_of_relevant_results/k

In [7]:
def mean_average_precision(max_k, base_hashtag, base_date, relevant_docs, docs_preprocessed):
  average_precision=0
  ctr=0
  for k_val in range(1,max_k+1):
    ctr+=1
    precision_at_k_val = precision_at_k(k_val, base_hashtag, base_date, relevant_docs, docs_preprocessed)
    #print('Hashtag: {}   Precision@{}: {}'.format(base_hashtag, k_val, precision_at_k_val))
    average_precision += precision_at_k_val
  return average_precision/ctr

In [8]:
def recall_at_k(k, base_hashtag, base_date, prediction_list, docs_preprocessed):
  relevant_docs_list = get_relevant_docs_list_for_base_hashtag(base_hashtag, base_date, docs_preprocessed)
  current_num_of_relevant_results=0
  for itr in range(k):
    if (prediction_list[itr] in relevant_docs_list):
      current_num_of_relevant_results+=1
  if(len(relevant_docs_list)==0):
    return 0
  return current_num_of_relevant_results/len(relevant_docs_list)

In [9]:
def mean_average_recall(max_k, base_hashtag, base_date, relevant_docs, docs_preprocessed):
  average_recall=0
  ctr=0
  for k_val in range(1,max_k+1):
    ctr+=1
    recall_at_k_val = recall_at_k(k_val, base_hashtag, base_date, relevant_docs, docs_preprocessed)
    #print('Hashtag: {}   Recall@{}: {}'.format(base_hashtag, k_val, recall_at_k_val))
    average_recall += recall_at_k_val
  return average_recall/ctr

In [10]:
def keyword_extractor(dataset):
  preprocessed_vocabulary = dict()

  #Converting to lowercase
  def to_lower_case(text):
    text = text.lower()
    return text

  def remove_at_word(text):
    data = text.split()
    data = [d for d in data if d[0]!='@']
    text = ' '.join(data)
    return text

  def remove_hashtag(text):
    data = text.split()
    data = [d if (d[0]!='#' or len(d) == 1) else d[1:] for d in data]
    data = [d for d in data if d[0]!='#']
    text = ' '.join(data)
    return text

  def remove_URL(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'bit.ly\S+', '', text, flags=re.MULTILINE)
    return text

  #Removing stopwords
  def remove_stopwords(text):
    stopword = stopwords.words('english')
    new_list = [x for x in text.split() if x not in stopword]
    return ' '.join(new_list)

  #Removing punctuations
  def remove_punctuations(text):
    punctuations = '''!()-[|]`{};:'"\,<>./?@#$=+%^&*_~'''
    new_list = ['' if x in punctuations else x for x in text.split()]
    new_list_final = []
    for token in new_list:
      new_token=""
      for char in token:
        if(char not in punctuations):
          new_token+=char
      if(len(new_token)!=0):
        new_list_final.append(new_token)
    return ' '.join(new_list_final)

  #Tokenization
  def tokenization(text):
    return word_tokenize(text)

  def pre_process(text):
    text = to_lower_case(text)
    text = remove_at_word(text)
    text = remove_hashtag(text)
    text = remove_URL(text)
    text = remove_stopwords(text)
    text = remove_punctuations(text)
    text = tokenization(text)
    for token in text:
      if token in preprocessed_vocabulary.keys():
        preprocessed_vocabulary[token] += 1
      else:
        preprocessed_vocabulary[token] = 1
    return text
  
  preprocessed_data = [pre_process(text) for text in dataset]

  #print(preprocessed_vocabulary)

  AOF_coefficient = sum(preprocessed_vocabulary.values())/len(preprocessed_vocabulary)
  vocabulary = {token.strip():preprocessed_vocabulary[token] for token in preprocessed_vocabulary.keys() if preprocessed_vocabulary[token] > AOF_coefficient and len(token.strip())}

  #print(vocabulary)

  final_tokens_per_tweet = []
  for data in preprocessed_data:
    final_tokens_per_tweet.append([token for token in data if token in vocabulary.keys()])

  #print(preprocessed_data)
  #print(final_tokens_per_tweet)

  word2id = dict()
  id2word = dict()
  vocabulary_size = len(vocabulary)
  count = 0
  for token in vocabulary.keys():
    word2id[token] = count
    id2word[count] = token
    count += 1

  #print(word2id)
  #print(id2word)

  directed_graph_adjacency_matrix = np.zeros((vocabulary_size, vocabulary_size))
  edge_weight_matrix = np.zeros((vocabulary_size, vocabulary_size))
  first_frequency = dict()
  last_frequency = dict()
  term_frequency = vocabulary
  strength = dict()
  degree = dict()
  selective_centraility = dict()


  for tweet in final_tokens_per_tweet:

    if tweet[0] in first_frequency.keys():
      first_frequency[tweet[0]] += 1
    else:
      first_frequency[tweet[0]] = 1

    if tweet[-1] in last_frequency.keys():
      last_frequency[tweet[-1]] += 1
    else:
      last_frequency[tweet[-1]] = 1
    


    for i in range(len(tweet)-1):
      if tweet[i] == tweet[i+1]:
        continue
      x = word2id[tweet[i]]
      y = word2id[tweet[i+1]]
      directed_graph_adjacency_matrix[x][y] += 1

  for tweet in final_tokens_per_tweet:
    for i in range(len(tweet)-1):


      if tweet[i] == tweet[i+1]:
        continue
      x = word2id[tweet[i]]
      y = word2id[tweet[i+1]]

    # Updating degree..
      if tweet[i] in degree.keys():
        degree[tweet[i]] += 1
      else:
        degree[tweet[i]] = 1
        
      if tweet[i+1] in degree.keys():
        degree[tweet[i+1]] += 1
      else:
        degree[tweet[i+1]] = 1

      edge_weight_matrix[x][y] = directed_graph_adjacency_matrix[x][y]/(vocabulary[tweet[i]] + vocabulary[tweet[i+1]] - directed_graph_adjacency_matrix[x][y])

      if tweet[i] in strength.keys():
        strength[tweet[i]] += edge_weight_matrix[x][y]
      else:
        strength[tweet[i]] = edge_weight_matrix[x][y]




  first_frequency = {token:(first_frequency[token]/vocabulary[token] if token in first_frequency else 0) for token in vocabulary.keys()}
  last_frequency = {token:(last_frequency[token]/vocabulary[token] if token in last_frequency else 0) for token in vocabulary.keys()}
  degree = {token:(degree[token] if token in degree else 0) for token in vocabulary.keys()}
  strength = {token:(strength[token] if token in strength else 0) for token in vocabulary.keys()}
  selective_centraility = {token:(strength[token]/degree[token] if degree[token]!=0 else 0) for token in vocabulary.keys()}

  #print(degree)
  #print(vocabulary)

  maxdegree = max(degree.items(), key=lambda x: x[1])[1]
  max_degree_nodes_with_freq = {key:term_frequency[key] for key in degree.keys() if degree[key] == maxdegree}
  maxfreq = max(max_degree_nodes_with_freq.items(), key=lambda x: x[1])[1]
  central_node_name = [key for key in max_degree_nodes_with_freq.keys() if max_degree_nodes_with_freq[key] == maxfreq][0]
  #print("central node: ", central_node_name)

  # bfs
  distance_from_central_node = dict()
  central_node_id = word2id[central_node_name]
  q = [(central_node_id, 0)]

  # Set source as visited
  distance_from_central_node[central_node_name] = 0

  while q:
      vis = q[0]
      # Print current node
      #print(id2word[vis[0]], vis[1])
      q.pop(0)
        
      # For every adjacent vertex to
      # the current vertex
      for i in range(len(directed_graph_adjacency_matrix[vis[0]])):
          if (directed_graph_adjacency_matrix[vis[0]][i] == 1 and (id2word[i] not in distance_from_central_node.keys())):
              # Push the adjacent node
              # in the queue
              q.append((i, vis[1]+1))
              distance_from_central_node[id2word[i]] = vis[1]+1

  #print(distance_from_central_node)
  inverse_distance_from_central_node = {token:(1/distance_from_central_node[token] if token in distance_from_central_node and token != central_node_name else 0) for token in vocabulary.keys()}
  inverse_distance_from_central_node[central_node_name] = 1.0
  #print(inverse_distance_from_central_node)

  neighbour_importance = dict()

  for i in range(len(directed_graph_adjacency_matrix)):
    neighbours = set()

    # traversing outgoing edges
    for j in range(len(directed_graph_adjacency_matrix)):
      if i == j:
        continue
      if directed_graph_adjacency_matrix[i][j] > 0:
        neighbours.add(j)
    for j in range(len(directed_graph_adjacency_matrix)):
      if i == j:
        continue
      if directed_graph_adjacency_matrix[j][i] > 0:
          neighbours.add(j)
    if len(neighbours) != 0:
      neighbour_importance[id2word[i]] = sum([strength[id2word[j]] for j in neighbours])/len(neighbours)
    else:
      neighbour_importance[id2word[i]] = 0
      
  #print(neighbour_importance)

  unnormalized_node_weight = {node: (first_frequency[node] + last_frequency[node] + term_frequency[node] + selective_centraility[node] + inverse_distance_from_central_node[node] + neighbour_importance[node]) for node in vocabulary.keys()}
  max_node_weight = max(unnormalized_node_weight.items(), key=lambda x: x[1])[1]
  min_node_weight = min(unnormalized_node_weight.items(), key=lambda x: x[1])[1]
  #print("max node weight: ", max_node_weight, "min node weight: ", min_node_weight)
  normalized_node_weight = {node: ((unnormalized_node_weight[node] - min_node_weight)/(max_node_weight - min_node_weight) if max_node_weight != min_node_weight else unnormalized_node_weight[node]) for node in unnormalized_node_weight.keys()}
  #print("Unnormalized score: ", unnormalized_node_weight)
  #print("Normalized score: ", normalized_node_weight)

  damping_factor = 0.85
  relevance_of_node = {node: np.random.uniform(0,1,1)[0] for node in vocabulary.keys()}
  threshold = 0.000000001


  #print(relevance_of_node)

  count = 0
  while True:
    count += 1
    current_relevance_of_node = dict()
    for node in vocabulary.keys():
      outer_sum = 0
      node_idx = word2id[node]
      for j in range(len(directed_graph_adjacency_matrix)):
        if j == node_idx:
          continue
        if directed_graph_adjacency_matrix[j][node_idx] > 0:
          den_sum = 0
          for k in range(len(directed_graph_adjacency_matrix)):
            if k == j:
              continue
            den_sum += directed_graph_adjacency_matrix[j][k]
          outer_sum += ((directed_graph_adjacency_matrix[j][node_idx]/den_sum) * relevance_of_node[id2word[j]])
      current_relevance_of_node[node] = (1-damping_factor)*normalized_node_weight[node] + damping_factor*normalized_node_weight[node]*outer_sum
    

    # checking convergence..
    sq_error = sum([(current_relevance_of_node[node] - relevance_of_node[node])**2 for node in vocabulary.keys()])
    relevance_of_node = current_relevance_of_node
    if sq_error < threshold:
      break

  #print(relevance_of_node)
  #print(count)

  degree_centrality  = {node: 0 for node in vocabulary.keys()}

  if len(directed_graph_adjacency_matrix) > 1:
    for i in range(len(directed_graph_adjacency_matrix)):
      count = 0
      for j in range(len(directed_graph_adjacency_matrix)):
        if i == j:
          continue
        if directed_graph_adjacency_matrix[j][i] > 0:
          count += 1
      degree_centrality[id2word[i]] = count / (len(directed_graph_adjacency_matrix)-1)

  #print(degree_centrality)

  final_keyword_rank = [{'node': node, 'NE_rank': relevance_of_node[node], 'Degree': degree_centrality[node]} for node in vocabulary.keys()]

  #print("-----------")
  final_keyword_rank = sorted(final_keyword_rank, key = lambda i: (i['NE_rank'], i['Degree']), reverse = True)

  final_keywords = [keyword['node'] for keyword in final_keyword_rank]

  return final_keywords

In [11]:
# ## Import the data ad create the inverted index

def import_dataset():
    """
    This function import all the articles in the TIME corpus,
    returning list of lists where each sub-list contains all the
    terms present in the document as a string.
    """
    # articles = []
    # with open('TIME.ALL', 'r') as f:
    #     tmp = []
    #     for row in f:
    #         if row.startswith("*TEXT"):
    #             if tmp != []:
    #                 articles.append(tmp)
    #             tmp = []
    #         else:
    #             row = re.sub(r'[^a-zA-Z\s]+', '', row)
    #             tmp += row.split()
    # return articles

    docs_preprocessed_metrics = []
    docs_preprocessed = []
    docs_preprocessed_with_names = []
    path = '/content/drive/MyDrive/Tweelink_Dataset/Tweelink_Articles_Processed'
    for filename in glob(os.path.join(path, '*')):
      with open(os.path.join(os.getcwd(), filename), 'r', encoding = 'utf-8',errors = 'ignore') as f:
        filename = os.path.basename(f.name)
        data = json.load(f)
        d_date = data["Date"]
        if(d_date=="" or d_date=="Date"):
          continue
        format = '%Y-%m-%d'
    
        d_present_date = datetime.datetime.strptime(d_date, format)
    
        if(str(d_present_date.date()) not in [str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())]):
          continue
      
        docs_preprocessed_metrics.append({'Name':filename, 'Data':data})
        docs_preprocessed.append(data['Body_processed'])
        docs_preprocessed_with_names.append(filename)

    return docs_preprocessed, docs_preprocessed_with_names, docs_preprocessed_metrics




# def remove_stop_words(corpus):
#    '''
#    This function removes from the corpus all the stop words present in the file TIME.STP
#    '''
#    stop_w = [line.rstrip('\n') for line in open('TIME.STP')]
#    stop_w=list(filter(None, stop_w))
#    for i in range(0,len(corpus)):
#        corpus[i] = [x for x in corpus[i] if x not in stop_w]
#    return corpus 


def make_inverted_index(corpus):
    """
    This function builds an inverted index as an hash table (dictionary)
    where the keys are the terms and the values are ordered lists of
    docIDs containing the term.
    """
    # corpus = remove_stop_words(corpus)
    index = defaultdict(set)
    for docid, article in enumerate(corpus):
        for term in article:
            index[term].add(docid)
    return index


# ### Union of two posting lists


def posting_lists_union(pl1, pl2):
        """
        Returns a new posting list resulting from the union of the
        two lists passed as arguments.
        """
        pl1 = sorted(list(pl1))
        pl2 = sorted(list(pl2))
        union = []
        i = 0
        j = 0
        while (i < len(pl1) and j < len(pl2)):
            if (pl1[i] == pl2[j]):
                union.append(pl1[i])
                i += 1
                j += 1
            elif (pl1[i] < pl2[j]):
                union.append(pl1[i])
                i += 1
            else:
                union.append(pl2[j])
                j += 1
        for k in range(i, len(pl1)):
            union.append(pl1[k])
        for k in range(j, len(pl2)):
            union.append(pl2[k])
        return union


# ## Precomputing weights


def DF(term, index):
    '''
    Function computing Document Frequency for a term.
    '''
    return len(index[term])


def IDF(term, index, corpus):
    '''
    Function computing Inverse Document Frequency for a term.
    '''
    return log(len(corpus)/DF(term, index))


def RSV_weights(corpus,index):
    '''
    This function precomputes the Retrieval Status Value weights
    for each term in the index
    '''
    N = len(corpus)
    w = {}
    for term in index.keys():
        p = DF(term, index)/(N+0.5)  
        w[term] = IDF(term, index, corpus) + log(p/(1-p))
    return w
    


# ## BIM Class


class BIM():
    '''
    Binary Independence Model class
    '''
    
    def __init__(self, corpus, articles_with_name):
        self.original_corpus = deepcopy(corpus)
        self.articles_with_name = articles_with_name
        self.articles = corpus
        self.index = make_inverted_index(self.articles)
        self.weights = RSV_weights(self.articles, self.index)
        self.ranked = []
        self.query_text = ''
        self.N_retrieved = 0
    
        
    def RSV_doc_query(self, doc_id, query):
        '''
        This function computes the Retrieval Status Value for a given couple document - query
        using the precomputed weights
        '''
        score = 0
        doc = self.articles[doc_id]
        for term in doc:
            if term in query:
                score += self.weights[term]     
        return score

    
        
    def ranking(self, query):
        '''
        Auxiliary function for the function answer_query. Computes the score only for documents
        that are in the posting list of al least one term in the query
        '''

        docs = []
        for term in self.index: 
            # print(term in query, query, term)
            if term in query:
                docs = posting_lists_union(docs, self.index[term])

        scores = []
        for doc in docs:
            scores.append((doc, self.RSV_doc_query(doc, query)))
        
        self.ranked = sorted(scores, key=lambda x: x[1], reverse = True)
        return self.ranked
    
    
    
    def recompute_weights(self, relevant_idx, query):
        '''
        Auxiliary function for relevance_feedback function and
        for the pseudo relevance feedback in answer_query function.
        Recomputes the weights, only for the terms in the query
        based on a set of relevant documents.
        '''
        
        relevant_docs = []
        for idx in relevant_idx:
            doc_id = self.ranked[idx-1][0]
            relevant_docs.append(self.articles[doc_id])
        
        N = len(self.articles)
        N_rel = len(relevant_idx)
        
        for term in query:
            if term in self.weights.keys():
                vri = 0
                for doc in relevant_docs:
                    if term in doc:
                        vri += 1
                p = (vri + 0.5) /( N_rel + 1)
                u = (DF(term, self.index) - vri + 0.5) / (N - N_rel +1)
                self.weights[term] = log((1-u)/u) + log(p/(1-p))

            
    
    def answer_query(self, query_text):
        '''
        Function to answer a free text query. Shows the first 30 words of the
        15 most relevant documents. 
        Also implements the pseudo relevance feedback with k = 5
        '''
        
        self.query_text = query_text
        query =  query_text.lower().split()
        ranking = self.ranking(query)

        ## pseudo relevance feedback 
        i = 0
        new_ranking=[]
        while i<10 and ranking != new_ranking:
            self.recompute_weights([1,2,3,4,5], query)
            new_ranking = self.ranking(query)
            i+=1


        ranking = new_ranking
        
        self.N_retrieved = 20
        
        ## print retrieved documents
        ret_list = []
        for i in range(0, self.N_retrieved):
            article = self.original_corpus[ranking[i][0]]
            if (len(article) > 30):
                article = article[0:30]
            text = " ".join(article)
            print(f"Article {self.articles_with_name[self.ranked[i][0]]}, score: {ranking[i][1]}")
            ret_list.append(self.articles_with_name[self.ranked[i][0]])
            # print(text, '\n')

        self.weights = RSV_weights(self.articles, self.index)
        return ret_list


            
    def relevance_feedback(self, *args):
        '''
        Function that implements relevance feedback for the last query formulated.
        The set of relevant documents is given by the user through a set of indexes
        '''
        if(self.query_text == ''):
            print('Cannot get feedback before a query is formulated.')
            return
        
        relevant_idx = list(args)
        
        if(isinstance(relevant_idx[0], list)):
            relevant_idx = relevant_idx[0]
        
        query = self.query_text.upper().split()
        self.recompute_weights(relevant_idx,query)
        
        self.answer_query(self.query_text)
    
    
    def read_document(self,rank_num):
        '''
        Function that allows the user to select a document among the ones returned 
        by answer_query and read the whole text
        '''
        if(self.query_text == ''):
            print('Cannot select a document before a query is formulated.')
            return
            
        article = self.original_corpus[self.ranked[rank_num - 1][0]]
        text = " ".join(article)
        print(f"Article {rank_num}, score: {self.ranked[rank_num][1]}")
        print(text, '\n')
        
        
    def show_more(self):
        '''
        Function that allows the user to see more 10 retrieved documents
        '''
        
        if(self.N_retrieved + 10 > len(self.ranked)):
            print('No more documents available')
            return 
        
        for i in range(self.N_retrieved, self.N_retrieved+10):
            article = self.original_corpus[self.ranked[i][0]]
            if (len(article) > 30):
                article = article[0:30]
            text = " ".join(article)
            print(f"Article {self.articles_with_name[self.ranked[i][0]]}, score: {self.ranked[i][1]}")
            print(text, '\n')
        
        self.N_retrieved += 10 
        



# Example of usage

# articles, articles_with_name = import_dataset()
# bim  = BIM(articles, articles_with_name)
# bim.answer_query('Italy and Great Britain fight the enemy')
# bim.relevance_feedback(5,6,8)
# bim.show_more()
# bim.read_document(2)

In [12]:
u_base_hashtag = input("Enter base hashtag: ")
u_time = input("Enter time: ")
u_location = input("Enter Location: ")

Enter base hashtag: narendramodi
Enter time: 2022-02-20
Enter Location: india


In [13]:
tweet_query = []
format = '%Y-%m-%d'
u_present_date = datetime.datetime.strptime(u_time, format)
u_prev_date = u_present_date - datetime.timedelta(days=1)
u_next_date = u_present_date + datetime.timedelta(days=1)
df_query = df.loc[df['hashtags'].str.contains(u_base_hashtag) & df['Date_Only'].isin([str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())])]

In [14]:
for tweet in df_query['Preprocessed_Data']:
  tweet_query.extend(tweet)

In [15]:
keyword_dataset = df_query['tweet'].tolist()
tweet_query_keyword_extractor = keyword_extractor(keyword_dataset)

In [16]:
tweet_keywords_yake = []
kw_extractor = yake.KeywordExtractor(top=20, stopwords=None)
keywords = kw_extractor.extract_keywords(' '.join(tweet_query))
#keywords = kw_extractor.extract_keywords(' '.join(df_query['tweet'].tolist()))
for kw, v in keywords:
  print("Keyphrase: ",kw, ": score", v)
  for key in kw.split():
    if(key.lower() not in tweet_keywords_yake):
      tweet_keywords_yake.append(key.lower())

print(tweet_keywords_yake)


Keyphrase:  india bjp modi : score 3.124693022092362e-05
Keyphrase:  narendramodi india bjp : score 6.037166731317872e-05
Keyphrase:  bjp india narendramodi : score 6.037166731317872e-05
Keyphrase:  india bjp narendramodi : score 6.037166731317872e-05
Keyphrase:  bjp narendramodi india : score 6.037166731317872e-05
Keyphrase:  modi india bjp : score 6.249386044184723e-05
Keyphrase:  bjp narendramodi congress : score 6.461576268648576e-05
Keyphrase:  congress bjp narendramodi : score 6.461576268648576e-05
Keyphrase:  india narendramodi modi : score 6.581080854383824e-05
Keyphrase:  narendramodi india modi : score 6.581080854383824e-05
Keyphrase:  priyankagandhi rahulgandhi congress : score 6.790842832286181e-05
Keyphrase:  bjp modi amitshah : score 6.851854888827196e-05
Keyphrase:  rahulgandhi congress bjp : score 6.94561200328927e-05
Keyphrase:  narendramodi modi congress : score 7.043825244061592e-05
Keyphrase:  india modi amitshah : score 7.487064473666578e-05
Keyphrase:  modi amitsh

In [17]:
# Example of usage
articles, articles_with_name, docs_preprocessed_metrics = import_dataset()

Plain Model (Binary Independence Model)

In [18]:
#results with normal query
bim  = BIM(articles, articles_with_name)
relevant_docs_plain = bim.answer_query(" ".join(tweet_query))

# bim.relevance_feedback(5,6,8)
# bim.show_more()
# bim.read_document(2)

print()

mean_average_precision_hashtag_bim_plain = mean_average_precision(20, u_base_hashtag, u_time, relevant_docs_plain, docs_preprocessed_metrics)
print('Mean Average Precision Plain Model (BIM) : {}'.format(mean_average_precision_hashtag_bim_plain))

mean_average_recall_hashtag_bim_plain = mean_average_recall(20, u_base_hashtag, u_time, relevant_docs_plain, docs_preprocessed_metrics)
print('Mean Average Recall Plain Model (BIM) : {}'.format(mean_average_recall_hashtag_bim_plain))

Article PunjabElections2022_127.json, score: 851.822652980454
Article PunjabElections2022_124.json, score: 447.1987543445336
Article narendramodi_305.json, score: 372.3686959203364
Article WriddhimanSaha_133.json, score: 311.070795969913
Article narendramodi_256.json, score: 306.0638537661412
Article PunjabElections2022_123.json, score: 255.53453475975095
Article PunjabElections2022_121.json, score: 240.12392229968475
Article narendramodi_303.json, score: 234.656032946595
Article narendramodi_307.json, score: 228.21348893511322
Article narendramodi_306.json, score: 228.02769184049515
Article narendramodi_304.json, score: 197.77697697389016
Article narendramodi_260.json, score: 183.86487484557077
Article PunjabElections2022_125.json, score: 182.33913582029274
Article PunjabElections2022_122.json, score: 172.5482501887701
Article nbaallstar_286.json, score: 172.11136229422772
Article hijab_286.json, score: 169.2184334596351
Article narendramodi_253.json, score: 165.32434684995448
Article

Model with Keyword Extractor (Binary Independence Model)

In [19]:
# results with top k keywords
bim  = BIM(articles, articles_with_name)
relevant_doce_yake = bim.answer_query(" ".join(tweet_keywords_yake))

# bim.relevance_feedback(5,6,8)
# bim.show_more()
# bim.read_document(2)

print()

mean_average_precision_hashtag_bim_keyword_extractor = mean_average_precision(20, u_base_hashtag, u_time, relevant_doce_yake, docs_preprocessed_metrics)
print('Mean Average Precision Keyword Extractor Model (BIM) : {}'.format(mean_average_precision_hashtag_bim_keyword_extractor))

mean_average_recall_hashtag_bim_keyword_extractor = mean_average_recall(20, u_base_hashtag, u_time, relevant_doce_yake, docs_preprocessed_metrics)
print('Mean Average Recall Keyword Extractor Model (BIM) : {}'.format(mean_average_recall_hashtag_bim_keyword_extractor))

Article PunjabElections2022_127.json, score: 237.17816728316586
Article narendramodi_253.json, score: 164.03729029589556
Article narendramodi_305.json, score: 154.9973112219869
Article narendramodi_306.json, score: 107.0378079638308
Article narendramodi_256.json, score: 95.14273664320255
Article narendramodi_260.json, score: 70.11842175694022
Article PunjabElections2022_121.json, score: 68.36557348119001
Article PunjabElections2022_123.json, score: 68.36557348119001
Article narendramodi_302.json, score: 67.50878296262857
Article UNWFPtosavesoil_153.json, score: 67.12815300153467
Article narendramodi_307.json, score: 67.03615261198105
Article hijab_286.json, score: 66.53316732419071
Article narendramodi_254.json, score: 66.34696509453492
Article narendramodi_303.json, score: 61.64186375019813
Article WriddhimanSaha_133.json, score: 56.7718189960597
Article PunjabElections2022_124.json, score: 53.06291345389182
Article narendramodi_304.json, score: 52.5969315051052
Article UNWFPtosavesoi

Model with YAKE (Binary Independence Model)

In [20]:
# results with top k keywords
bim  = BIM(articles, articles_with_name)
relevant_docs_yake = bim.answer_query(" ".join(tweet_keywords_yake))

# bim.relevance_feedback(5,6,8)
# bim.show_more()
# bim.read_document(2)

print()

mean_average_precision_hashtag_bim_yake = mean_average_precision(20, u_base_hashtag, u_time, relevant_docs_yake, docs_preprocessed_metrics)
print('Mean Average Precision Plain Model (BIM) : {}'.format(mean_average_precision_hashtag_bim_yake))

mean_average_recall_hashtag_bim_yake = mean_average_recall(20, u_base_hashtag, u_time, relevant_docs_yake, docs_preprocessed_metrics)
print('Mean Average Recall Plain Model (BIM) : {}'.format(mean_average_recall_hashtag_bim_yake))

Article PunjabElections2022_127.json, score: 237.17816728316586
Article narendramodi_253.json, score: 164.03729029589556
Article narendramodi_305.json, score: 154.9973112219869
Article narendramodi_306.json, score: 107.0378079638308
Article narendramodi_256.json, score: 95.14273664320255
Article narendramodi_260.json, score: 70.11842175694022
Article PunjabElections2022_121.json, score: 68.36557348119001
Article PunjabElections2022_123.json, score: 68.36557348119001
Article narendramodi_302.json, score: 67.50878296262857
Article UNWFPtosavesoil_153.json, score: 67.12815300153467
Article narendramodi_307.json, score: 67.03615261198105
Article hijab_286.json, score: 66.53316732419071
Article narendramodi_254.json, score: 66.34696509453492
Article narendramodi_303.json, score: 61.64186375019813
Article WriddhimanSaha_133.json, score: 56.7718189960597
Article PunjabElections2022_124.json, score: 53.06291345389182
Article narendramodi_304.json, score: 52.5969315051052
Article UNWFPtosavesoi

In [21]:
# global_list = [['narendramodi', '2022-02-14', 'india'],['UkraineRussiaCrisis', '2022-02-14', 'ukraine'],['IPL', '2022-02-14', 'india'],['TaylorSwift', '2022-02-14', 'USA'],['IndiaFightsCorona', '2022-02-14', 'india'],['narendramodi', '2022-02-15', 'india'],['UkraineRussiaCrisis', '2022-02-15', 'ukraine'],['IPL', '2022-02-15', 'india'],['TaylorSwift', '2022-02-15', 'USA'],['IndiaFightsCorona', '2022-02-15', 'india'],['narendramodi', '2022-02-16', 'india'],['UkraineRussiaCrisis', '2022-02-16', 'ukraine'],['IPL', '2022-02-16', 'india'],['TaylorSwift', '2022-02-16', 'USA'],['IndiaFightsCorona', '2022-02-16', 'india'],['narendramodi', '2022-02-17', 'india'],['UkraineRussiaCrisis', '2022-02-17', 'ukraine'],['IPL', '2022-02-17', 'india'],['TaylorSwift', '2022-02-17', 'USA'],['IndiaFightsCorona', '2022-02-17', 'india'],['narendramodi', '2022-02-18', 'india'],['UkraineRussiaCrisis', '2022-02-18', 'ukraine'],['IPL', '2022-02-18', 'india'],['TaylorSwift', '2022-02-18', 'USA'],['IndiaFightsCorona', '2022-02-18', 'india'],['narendramodi', '2022-02-19', 'india'],['UkraineRussiaCrisis', '2022-02-19', 'ukraine'],['IPL', '2022-02-19', 'india'],['hijab', '2022-02-19', 'india'],['vaccine', '2022-02-19', 'india'],['MillionAtIndiaPavilion', '2022-02-14', 'UAE'],['PunjabPanjeNaal', '2022-02-14', 'India'],['Euphoria', '2022-02-14', 'World'],['OscarsFanFavorite', '2022-02-14', 'World'],['ShameOnBirenSingh', '2022-02-14', 'india'],['BappiLahiri', '2022-02-16', 'india'],['BlandDoritos', '2022-02-16', 'USA'],['VERZUZ', '2022-02-16', 'USA'],['DragRaceUK', '2022-02-16', 'United Kingdom'],['BoycottWalgreens', '2022-02-18', 'USA'],['PunjabElections2022', '2022-02-20', 'india'],['WriddhimanSaha', '2022-02-20', 'india'],['stormfranklin', '2022-02-20', 'USA'],['QueenElizabeth', '2022-02-20', 'United Kingdom'],['ScottyFromWelding', '2022-02-20', 'Australia'],['CarabaoCupFinal', '2022-02-27', 'London'],['NZvSA', '2022-02-28', 'New Zealand'],['IPCC', '2022-02-28', 'Worldwide'],['SuperBowl', '2022-02-14', 'USA'],['MultiverseOfMadness', '2022-02-14', 'USA'],['Eminem', '2022-02-14', 'USA'],['IPLAuction', '2022-02-14', 'india'],['JohnsonOut21', '2022-02-14', 'United Kingdom'],['Cyberpunk2077', '2022-02-15', 'Worldwide'],['Wordle242', '2022-02-15', 'Worldwide'],['DeepSidhu', '2022-02-15', 'india'],['CanadaHasFallen', '2022-02-15', 'canada'],['IStandWithTrudeau', '2022-02-15', 'canada'],['CNNPHVPDebate', '2022-02-26', 'philippines'],['qldfloods', '2022-02-26', 'australia'],['Eurovision', '2022-02-26', 'worldwide'],['IndiansInUkraine', '2022-02-26', 'india'],['PritiPatel', '2022-02-26', 'united kingdom'],['TaylorCatterall', '2022-02-27', 'united kingdom'],['PSLFinal', '2022-02-27', 'pakistan'],['AustraliaDecides', '2022-02-27', 'australia'],['WorldNGODay', '2022-02-27', 'worldwide'],['TheBatman', '2022-02-28', 'USA'],['NationalScienceDay', '2022-02-28', 'india'],['msdtrong', '2022-02-14', 'india'],['Boycott_ChennaiSuperKings', '2022-02-14', 'india'],['GlanceJio', '2022-02-14', 'india'],['ArabicKuthu', '2022-02-14', 'india'],['Djokovic', '2022-02-15', 'australia'],['Real Madrid', '2022-02-15', 'santiago'],['bighit', '2022-02-15', 'korea'],['Maxwell', '2022-02-15', 'australia'],['mafsau', '2022-02-16', 'australia'],['channi', '2022-02-16', 'punjab'],['ayalaan', '2022-02-16', 'india'],['jkbose', '2022-02-16', 'india'],['HappyBirthdayPrinceSK', '2022-02-16', 'india'],['RandomActsOfKindnessDay', '2022-02-17', 'worldwide'],['happybirthdayjhope', '2022-02-17', 'korea'],['mohsinbaig', '2022-02-17', 'pakistan'],['aewdynamite', '2022-02-17', 'worldwide'],['aaraattu', '2022-02-17', 'india'],['ShivajiJayanti', '2022-02-18', 'india'],['PlotToKillModi', '2022-02-18', 'india'],['NationalDrinkWineDay', '2022-02-18', 'usa'],['HorizonForbiddenWest', '2022-02-18', 'worldwide'],['BoycottWalgreens', '2022-02-18', 'usa'],['CallTheMidwife', '2022-02-20', 'worldwide'],['OperationDudula', '2022-02-20', 'south africa'],['truthsocial', '2022-02-21', 'usa'],['nbaallstar', '2022-02-21', 'usa'],['shivamogga', '2022-02-21', 'india'],['HalftimeShow', '2022-02-14', 'usa'],['OttawaStrong', '2022-02-14', 'canada'],['DrDre', '2022-02-14', 'usa'],['BattleOfBillingsBridge', '2022-02-14', 'usa'],['FullyFaltooNFTdrop', '2022-02-14', 'worldwide'],['AK61', '2022-02-15', 'india'],['sandhyamukherjee', '2022-02-15', 'india'],['MUNBHA', '2022-02-15', 'worldwide'],['nursesstrike', '2022-02-15', 'australia'],['Realme9ProPlus', '2022-02-16', 'worldwide'],['KarnatakaHijabControversy', '2022-02-16', 'india'],['BJPwinningUP', '2022-02-16', 'india'],['Punjab_With_Modi', '2022-02-16', 'india'],['PushpaTheRule', '2022-02-16', 'india'],['RehmanMalik', '2022-02-22', 'india'],['harisrauf', '2022-02-22', 'pakistan'],['Rosettenville', '2022-02-22', 'south africa'],['NFU22', '2022-02-22', 'worldwide'],['justiceforharsha', '2022-02-22', 'india'],['wordle251', '2022-02-24', 'worldwide'],['ARSWOL', '2022-02-24', 'worldwide'],['stopwar', '2022-02-24', 'worldwide'],['PrayForPeace', '2022-02-24', 'worldwide'],['StopPutinNOW', '2022-02-24', 'worldwide'],['TeamGirlsCup', '2022-02-25', 'worldwide'],['Canucks', '2022-02-25', 'worldwide'],['PinkShirtDay', '2022-02-25', 'canada'],['superrugbypacific', '2022-02-25', 'australia']]

In [22]:
# global_average_mean_average_precision_1 = []
# global_mean_average_recall_1 = []

# global_average_mean_average_precision_2 = []
# global_mean_average_recall_2 = []

# for iter in tqdm(range(len(global_list))):
#   u_base_hashtag = global_list[iter][0]
#   u_time = global_list[iter][1]
#   u_location = global_list[iter][2]
#   tweet_query = []
#   format = '%Y-%m-%d'
#   u_present_date = datetime.datetime.strptime(u_time, format)
#   u_prev_date = u_present_date - datetime.timedelta(days=1)
#   u_next_date = u_present_date + datetime.timedelta(days=1)
#   df_query = df.loc[df['hashtags'].str.contains(u_base_hashtag) & df['Date_Only'].isin([str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())])]

#   for tweet in df_query['Preprocessed_Data']:
#     tweet_query.extend(tweet)
  
#   tweet_keywords = []
#   kw_extractor = yake.KeywordExtractor(top=20, stopwords=None)
#   keywords = kw_extractor.extract_keywords(' '.join(tweet_query))
#   for kw, v in keywords:
#     #print("Keyphrase: ",kw, ": score", v)
#     for key in kw.split():
#       if(key not in tweet_keywords):
#         tweet_keywords.append(key)
  
#   docs_preprocessed = []

#   total_documents = 0
#   path = '/content/drive/MyDrive/Tweelink_Dataset/Tweelink_Articles_Processed'
#   for filename in glob(os.path.join(path, '*')):
#     with open(os.path.join(os.getcwd(), filename), 'r', encoding = 'utf-8',errors = 'ignore') as f:
#       filename = os.path.basename(f.name)
#       data = json.load(f)
#       d_date = data["Date"]
#       if(d_date=="" or d_date=="Date"):
#         continue
#       format = '%Y-%m-%d'
  
#       d_present_date = datetime.datetime.strptime(d_date, format)
  
#       if(str(d_present_date.date()) not in [str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())]):
#         continue
    
#       docs_preprocessed.append({'Name':filename, 'Data':data})
#       total_documents+=1
  
#   articles, articles_with_name, docs_preprocessed_metrics = import_dataset()
#   bim  = BIM(articles, articles_with_name)
#   relevant_articles_list_1 = bim.answer_query(" ".join(tweet_query))
#   mean_average_precision_hashtag_1 = mean_average_precision(20, u_base_hashtag, u_time, relevant_articles_list_1, docs_preprocessed_metrics)
#   global_average_mean_average_precision_1.append(mean_average_precision_hashtag_1)
#   mean_average_recall_hashtag_1 = mean_average_recall(20, u_base_hashtag, u_time, relevant_articles_list_1, docs_preprocessed_metrics)
#   global_mean_average_recall_1.append(mean_average_recall_hashtag_1)

#   bim  = BIM(articles, articles_with_name)
#   relevant_articles_list_2 = bim.answer_query(" ".join(tweet_keywords))
#   mean_average_precision_hashtag_2 = mean_average_precision(20, u_base_hashtag, u_time, relevant_articles_list_2, docs_preprocessed_metrics)
#   global_average_mean_average_precision_2.append(mean_average_precision_hashtag_2)
#   mean_average_recall_hashtag_2 = mean_average_recall(20, u_base_hashtag, u_time, relevant_articles_list_2, docs_preprocessed_metrics)
#   global_mean_average_recall_2.append(mean_average_recall_hashtag_2)

In [23]:
# overall_average_mean_average_precision_1 = sum(global_average_mean_average_precision_1)/len(global_average_mean_average_precision_1)
# print(overall_average_mean_average_precision_1)

# overall_mean_average_recall_1 = sum(global_mean_average_recall_1)/len(global_mean_average_recall_1)
# print(overall_mean_average_recall_1)

In [24]:
# overall_average_mean_average_precision_2 = sum(global_average_mean_average_precision_2)/len(global_average_mean_average_precision_2)
# print(overall_average_mean_average_precision_2)

# overall_mean_average_recall_2 = sum(global_mean_average_recall_2)/len(global_mean_average_recall_2)
# print(overall_mean_average_recall_2)